In [1]:
import pandas as pd
import functools
from collections import defaultdict
from typing import List

from pybark import db, flatten_list_of_lists, genotype_frequencies, health, plink, read_plink_files, sapi, shell
from pybark.health import Probe, ProbeCall, HealthCondition


def get_biodash_breed_data_for_low_imiss_chip_dogs(chip_version_number):
    breed_df = pd.DataFrame(
        db.run_query(
            ENVIRONMENT="prod",
            sql=f"SELECT genotype_id, swab_code, overall_missingness, breed1pct, breed1code, CONCAT_WS(',', breed1code, breed2code, breed3code, breed4code, breed5code, breed6code, breed7code, breed8code) AS all_breedcodes FROM genotypes WHERE breed1code IS NOT NULL AND overall_missingness <= 0.02 AND chip_version_number = {chip_version_number}",
        )
    ).fillna("NA")
    breed_df["breed_mix_scenario"] = breed_df.apply(
        lambda row: "purebred" if float(row["breed1pct"] > 95.) else "mix", axis=1,
    )
    return breed_df


def get_dogs_with_bioinformatics_breed_label_df(
    bioinformatics_breed_label,
    per_dog_breed_ancestry_data_df,
):
    breed_filter = per_dog_breed_ancestry_data_df.apply(
        lambda row: bioinformatics_breed_label in row["all_breedcodes"].split(","), axis=1
    )
    has_breed_df = per_dog_breed_ancestry_data_df.loc[
        breed_filter, ["genotype_id", "all_breedcodes"],
    ]
    return has_breed_df[["genotype_id"]]


def get_bool_on_target_breed_ancestry_in_dog_breed_data_df(
    on_target_breed_list,
    per_dog_breed_ancestry_data_df
):
    any_on_target_breed_bool_df = pd.DataFrame()
    for breed in on_target_breed_list:
        try:
            has_breed_df = get_dogs_with_bioinformatics_breed_label_df(
                bioinformatics_breed_label=breed,
                per_dog_breed_ancestry_data_df=per_dog_breed_ancestry_data_df,
            )
            print(f"breed: {breed}, has_breed: {has_breed_df.shape[0]}")
            any_on_target_breed_bool_df = any_on_target_breed_bool_df.append(
                has_breed_df, ignore_index=True, sort=False,
            )
        except ValueError:
            pass
    # drop duplicate genotype_ids (from mixed breed dogs with multiple on-target breeds)
    any_on_target_breed_bool_df.drop_duplicates(inplace=True)
    any_on_target_breed_bool_df["has_on_target_breed_ancestry"] = "True"
    return any_on_target_breed_bool_df


def get_GTs_with_on_breed_bool_df_for_health_id(
    health_id,
    health_id_sub_df_with_breed,
    bioinformatics_breed_labels,
    health_id_on_target_breed_dict,
):
    try:
        good_on_target_breed_list = [
            b for b in health_id_on_target_breed_dict[str(health_id)] if b in bioinformatics_breed_labels
        ]
        print(
            f"{health_id} on-target breed list: {good_on_target_breed_list}"
        )
        any_on_target_breed_bool_df = (
            get_bool_on_target_breed_ancestry_in_dog_breed_data_df(
                on_target_breed_list=good_on_target_breed_list,
                per_dog_breed_ancestry_data_df=health_id_sub_df_with_breed,
            )
        )
        print(f"{any_on_target_breed_bool_df.shape[0]} unique dogs with any on-target breed ancestry")
        final_health_id_sub_df = health_id_sub_df_with_breed.merge(
            any_on_target_breed_bool_df, on="genotype_id", how="left",
        ).fillna("False")
    except KeyError:
        print(f"No on-target breed list available for {health_id}")
        final_health_id_sub_df = health_id_sub_df_with_breed
        final_health_id_sub_df["has_on_target_breed_ancestry"] = "not_applicable"
    final_health_id_sub_df["health_id_n_dogs"] = final_health_id_sub_df.shape[0]
    return final_health_id_sub_df


def get_probe_objects_from_df(probes_df):
        return [
            Probe(row.probe, row.at_risk_allele, row.clear_allele)
            for probe, row in probes_df.iterrows()
        ]



Requested attributes (per https://docs.google.com/document/d/1yj6uqCmY4T8ix38aCX73OpWiOkjGU2AJLcGGTXBPWPM/edit)

From pets table:

Dog swab_code
Dog pet_id
Dog name (per pets table)
Customer name
Customer account type (breeder, pet owner, vet)
Customer email address
Test name
Kit type (B+H, Breed ID, Purebred, Breeder)
Dog profile has traits/health unlocked (original purchase or upgrade) (Y/N)
Dog marked as deceased (Y/N)
Customer has consented to comms concerning this dog (Y/N)
Dog intact status (I = intact, N = neutered)
Owner has more than 1 dogs that would be receiving updates (Y/N)

From this code:
Current test result on customer site
New test result (to be backfilled)
Dog has on target breed ancestry (Y/N)
Purebred (Y/N)


In [2]:
# specify new probe data
new_health_id_probe_df = pd.DataFrame({
    "health_id": "070000",
    "probe": ["EMB_chr13_59946494", "EMB_chr13_59946494_a_new", "EMB_chr13_59946494_dup", "EMB_chr13_59946494a", "EMB_chr13_59946494a_dup", "EMB_chr13_59946494a_dup2"],
    "at_risk_allele": ["D", "G", "D", "D", "D", "D"],
    "clear_allele": ["I", "A", "I", "I", "I", "I"],
    "chip_version_numbers": [[5], [5], [5], [3, 4, 5], [1, 2, 3, 4, 5], [2, 3, 4, 5]]
})

new_health_id_probe_df_dict = defaultdict(list)
for probe, probe_df in new_health_id_probe_df.groupby("probe"):
    eligible_chip_versions = probe_df["chip_version_numbers"].values[0]
    for chip_version_number in eligible_chip_versions:
        new_health_id_probe_df_dict[chip_version_number] = new_health_id_probe_df_dict[chip_version_number] + [probe]

new_health_id_probe_df_dict

defaultdict(list,
            {5: ['EMB_chr13_59946494',
              'EMB_chr13_59946494_a_new',
              'EMB_chr13_59946494_dup',
              'EMB_chr13_59946494a',
              'EMB_chr13_59946494a_dup',
              'EMB_chr13_59946494a_dup2'],
             3: ['EMB_chr13_59946494a',
              'EMB_chr13_59946494a_dup',
              'EMB_chr13_59946494a_dup2'],
             4: ['EMB_chr13_59946494a',
              'EMB_chr13_59946494a_dup',
              'EMB_chr13_59946494a_dup2'],
             1: ['EMB_chr13_59946494a_dup'],
             2: ['EMB_chr13_59946494a_dup', 'EMB_chr13_59946494a_dup2']})

In [3]:
# fetch current health_state values for all dogs from biodash
old_health_state_df = pd.DataFrame(
    db.run_query(
        ENVIRONMENT="prod",
        sql=f"SELECT genotype_id, health_state FROM genotype_health WHERE health_id = '070000'")
).fillna("NA").rename(columns={"health_state": "health_state_old"})


In [4]:
# generate results using new probes (use local copies of health probe sheets)
on_target_breed_dict = {"070000": ["italian_greyhound"]}
all_conditions_df = sapi.get_health_conditions_df()
conditions_df = all_conditions_df[all_conditions_df["health_id"].isin(["070000"]) & (all_conditions_df[f"is_validated_for_v{chip_version_number}"])]

all_chip_version_new_health_states_df = pd.DataFrame()

for chip_version_number in range(1, 6):
    
    chip_probes_list = new_health_id_probe_df_dict[chip_version_number]
    
    # Filter allGenotypesAllMarkers to these dogs at the relevant probes
    plink.run_plink_with_options(
        [
            "--bfile", "/Users/andreaslavney/Desktop/allGenotypesAllMarkers/allGenotypesAllMarkers",
            "--snps", f"{','.join(chip_probes_list)}",
            "--make-bed",
            "--out", f"./v{chip_version_number}_probes_for_070000",
        ]
    )
    
    # fetch breed info from biodash
    breed_df = get_biodash_breed_data_for_low_imiss_chip_dogs(chip_version_number)
    print(f"{breed_df.shape[0]} dogs run on chip v{chip_version_number}")
    bioinformatics_breed_labels = list(
        set(",".join(breed_df["all_breedcodes"].dropna().tolist()).split(","))
    )
    
    # extract genotypes for these dogs at the new markers
    breed_df[["genotype_id", "genotype_id"]].to_csv(f"v{chip_version_number}_keepfile.txt", sep=' ', index=False, header=False)
    plink.run_plink_with_options(
        [
            "--bfile", f"./v{chip_version_number}_probes_for_070000",
            "--keep", f"v{chip_version_number}_keepfile.txt",
            "--recode", "tab",
            "--out", f"./v{chip_version_number}_probes_for_070000",
        ]
    )

    ped_df = read_plink_files.get_ped_df(
        plink_file_basename=f"./v{chip_version_number}_probes_for_070000"
    )
    
    # get health calls using new probes with core pybark.health code
    Probes = get_probe_objects_from_df(new_health_id_probe_df[new_health_id_probe_df["probe"].isin(chip_probes_list)])    
    conditions_df["probes"] = [Probes]
    health_conditions = [HealthCondition.from_series(row) for _, row in conditions_df.iterrows()]
    health_states_df = health.get_calls_for_health_conditions(
        ped_df=ped_df, health_conditions=health_conditions
    )
    health_calls_df = health_states_df.melt(
        id_vars="genotype_id", var_name="health_id", value_name="health_state"
    ).rename(columns={"health_state": "health_state_new"})
    
    # merge breed info with, new health_state, old_health_state data
    merge_health_calls_breed_df = health_calls_df.merge(breed_df, on="genotype_id")
    
    merge_health_calls_breed_bool_df = get_GTs_with_on_breed_bool_df_for_health_id(
        health_id="070000",
        health_id_sub_df_with_breed=merge_health_calls_breed_df,
        bioinformatics_breed_labels=bioinformatics_breed_labels,
        health_id_on_target_breed_dict=on_target_breed_dict,
    )
    
    final_merged_data_df = merge_health_calls_breed_bool_df.merge(old_health_state_df, on="genotype_id")
    #display(final_merged_data_df.head())
    
    all_chip_version_new_health_states_df = all_chip_version_new_health_states_df.append(final_merged_data_df, ignore_index=True)
    print()
    
all_chip_version_new_health_states_df[["genotype_id", "health_id", "swab_code", "health_state_old", "health_state_new", "breed_mix_scenario", "has_on_target_breed_ancestry"]]
    

2972 dogs run on chip v1


/Users/andreaslavney/opt/miniconda3/envs/pipeline/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


070000 on-target breed list: ['italian_greyhound']
breed: italian_greyhound, has_breed: 5
5 unique dogs with any on-target breed ancestry

36929 dogs run on chip v2


/Users/andreaslavney/opt/miniconda3/envs/pipeline/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


070000 on-target breed list: ['italian_greyhound']
breed: italian_greyhound, has_breed: 33
33 unique dogs with any on-target breed ancestry

114913 dogs run on chip v3


...

/Users/andreaslavney/opt/miniconda3/envs/pipeline/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


070000 on-target breed list: ['italian_greyhound']
breed: italian_greyhound, has_breed: 258
258 unique dogs with any on-target breed ancestry

497761 dogs run on chip v5


/Users/andreaslavney/opt/miniconda3/envs/pipeline/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


070000 on-target breed list: ['italian_greyhound']
breed: italian_greyhound, has_breed: 417
417 unique dogs with any on-target breed ancestry



,genotype_id,health_id,swab_code,health_state_old,health_state_new,breed_mix_scenario,has_on_target_breed_ancestry
0,WG0240808-DNA_A02_15624,070000,15624,2,2,purebred,False
1,WG0240808-DNA_A03_16173,070000,16173,2,2,purebred,False
2,WG0240808-DNA_A04_16248,070000,16248,2,2,purebred,False
3,WG0240808-DNA_A05_18187,070000,18187,2,2,purebred,False
4,WG0240808-DNA_A06_18213,070000,18213,2,2,purebred,False
...,...,...,...,...,...,...,...
909555,WG0525585-DNA_H08_31201151706869,070000,31201151706869,2,2,mix,False
909556,WG0525585-DNA_H09_31210351407989,070000,31210351407989,2,2,mix,False
909557,WG0525585-DNA_H10_31210452801041,070000,31210452801041,2,2,mix,False
909558,WG0525585-DNA_H11_31019081602461,070000,31019081602461,2,2,mix,False


In [5]:
# get info from pets table (have to go thru metabase)

# https://metabase.embarkvet.com/question/873-all-customers-that-can-be-contacted-about-backfilled-health-results -> query_result_2021-12-02T19_34_18.210441Z.csv
pets_info_df = pd.read_csv("query_result_2021-12-02T19_34_18.210441Z.csv")
pets_info_df_cols_to_keep = ["user_id", "email", "is_breeder", "pet_name", "active_swab_code", "state", "is_originally_comprehensive", "comprehensive_upgrade_purchased_at", "is_purebred_product"]

health_state_cols_to_keep = ["genotype_id", "health_id", "health_state_old", "health_state_new", "breed_mix_scenario", "has_on_target_breed_ancestry"]
                             
all_merged_data_df = all_chip_version_new_health_states_df.merge(pets_info_df, on="genotype_id")
all_merged_data_df["requires_update"] = all_merged_data_df.apply(lambda row: 'True' if row["health_state_old"] != row["health_state_new"] else 'False', axis=1)
all_merged_data_df[pets_info_df_cols_to_keep + health_state_cols_to_keep + ["requires_update"]].to_csv("backfill_data_070000_20211202.csv", index=False)
all_merged_data_df[pets_info_df_cols_to_keep + health_state_cols_to_keep + ["requires_update"]]


,user_id,email,is_breeder,pet_name,active_swab_code,state,is_originally_comprehensive,comprehensive_upgrade_purchased_at,is_purebred_product,genotype_id,health_id,health_state_old,health_state_new,breed_mix_scenario,has_on_target_breed_ancestry,requires_update
0,208159,safiradrak@gmail.com,False,Una z Fešandy,15624,analysisCompleted,True,NaN,False,WG0240808-DNA_A02_15624,070000,2,2,purebred,False,False
1,208159,safiradrak@gmail.com,False,Andora Gothywen,16173,analysisCompleted,True,NaN,False,WG0240808-DNA_A03_16173,070000,2,2,purebred,False,False
2,208159,safiradrak@gmail.com,False,Apollo z Julčina dvora,16248,analysisCompleted,True,NaN,False,WG0240808-DNA_A04_16248,070000,2,2,purebred,False,False
3,208159,safiradrak@gmail.com,False,Cira z Včelínského lesa,18187,analysisCompleted,True,NaN,False,WG0240808-DNA_A05_18187,070000,2,2,purebred,False,False
4,208159,safiradrak@gmail.com,False,Lucka z Práchové,18213,analysisCompleted,True,NaN,False,WG0240808-DNA_A06_18213,070000,2,2,purebred,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670803,740115,chris.j.haray@gmail.com,False,George,31201051306190,analysisCompleted,True,NaN,False,WG0525585-DNA_H01_31201051306190,070000,2,2,mix,False,False
670804,150425,blaisevitale@gmail.com,False,Ryder,31201053009573,analysisCompleted,False,2021-11-01T17:48:34.193598-04:00,False,WG0525585-DNA_H03_31201053009573,070000,2,2,mix,False,False
670805,739299,szahra7@gmail.com,False,Duke,31201152618034,analysisCompleted,True,NaN,False,WG0525585-DNA_H05_31201152618034,070000,2,2,purebred,False,False
670806,737416,adamsobetski@yahoo.com,False,Harley,31210751901334,analysisCompleted,True,NaN,False,WG0525585-DNA_H06_31210751901334,070000,2,2,mix,False,False


In [6]:
# summarize changes in health_state binned by breed info
summary_df = pd.DataFrame()
for name, group_df in all_merged_data_df.groupby(["health_state_old", "health_state_new", "has_on_target_breed_ancestry", "breed_mix_scenario", "is_breeder", "requires_update"]):
    summary_df = summary_df.append(
        {
            "requires_update": name[5],
            "health_state_old": str(int(name[0])),
            "health_state_new": str(int(name[1])),
            "has_on_target_breed_ancestry": name[2],
            "breed_mix_scenario": name[3],
            "is_breeder": name[4],
            "n_dogs": group_df.shape[0],
        }, ignore_index=True
    )

summary_df[["requires_update", "health_state_old", "health_state_new", "has_on_target_breed_ancestry", "breed_mix_scenario", "is_breeder", "n_dogs"]].sort_values(
    by=["requires_update", "health_state_old", "health_state_new", "has_on_target_breed_ancestry", "breed_mix_scenario", "is_breeder", "n_dogs"], ascending=False).to_csv(
    "backfill_data_070000_summary_20211202.csv", index=False
)
summary_df[["requires_update", "health_state_old", "health_state_new", "has_on_target_breed_ancestry", "breed_mix_scenario", "is_breeder", "n_dogs"]].sort_values(
    by=["requires_update", "health_state_old", "health_state_new", "has_on_target_breed_ancestry", "breed_mix_scenario", "is_breeder", "n_dogs"], ascending=False)


,requires_update,health_state_old,health_state_new,has_on_target_breed_ancestry,breed_mix_scenario,is_breeder,n_dogs
23,True,6,3,True,mix,0.0,1.0
22,True,6,2,False,purebred,1.0,1.0
21,True,6,2,False,purebred,0.0,14.0
20,True,6,2,False,mix,1.0,1.0
19,True,6,2,False,mix,0.0,15.0
12,True,3,2,False,purebred,1.0,34.0
11,True,3,2,False,purebred,0.0,167.0
10,True,3,2,False,mix,1.0,44.0
9,True,3,2,False,mix,0.0,1159.0
0,True,1,2,False,mix,0.0,2.0


In [21]:
multi_dog_owner_summary_df = pd.DataFrame()
for user_id, user_id_df in all_merged_data_df[all_merged_data_df["requires_update"]=='True'].groupby("user_id"):
    if user_id_df.shape[0] > 1:
        print(user_id_df[["user_id", "is_breeder", "health_state_old", "health_state_new", "has_on_target_breed_ancestry", "breed_mix_scenario"]])
        for category, category_df in user_id_df.groupby(["is_breeder", "health_state_old", "health_state_new", "has_on_target_breed_ancestry", "breed_mix_scenario"]):
            multi_dog_owner_summary_df = multi_dog_owner_summary_df.append(
                {
                    "user_id": int(user_id),
                    "email": user_id_df["email"].unique()[0],
                    "is_breeder": category[0],
                    "health_state_old": category[1],
                    "health_state_new": category[2],
                    "has_on_target_breed_ancestry": category[3],
                    "breed_mix_scenario": category[4],
                    "n_dogs": category_df.shape[0],
                }, ignore_index=True,
            )

print(f"{len(multi_dog_owner_summary_df['user_id'].unique())} users have >1 dog that will need to be updated")
print(f"of these, {multi_dog_owner_summary_df.drop_duplicates(subset=['user_id'])['is_breeder'].sum()} have breeder profiles")
display(multi_dog_owner_summary_df[["user_id", "email", "is_breeder", "has_on_target_breed_ancestry", "breed_mix_scenario", "health_state_old", "health_state_new", "n_dogs"]])
for c, c_df in multi_dog_owner_summary_df.groupby(["has_on_target_breed_ancestry", "breed_mix_scenario", "health_state_old", "health_state_new"]):
    print(c, c_df.shape[0])
        

        user_id  is_breeder  health_state_old  health_state_new  \
248208     3548       False                 3                 2   
248948     3548       False                 3                 2   

       has_on_target_breed_ancestry breed_mix_scenario  
248208                        False                mix  
248948                        False                mix  
        user_id  is_breeder  health_state_old  health_state_new  \
8005       4712        True                 3                 2   
11383      4712        True                 3                 2   
139462     4712        True                 3                 2   

       has_on_target_breed_ancestry breed_mix_scenario  
8005                          False           purebred  
11383                         False                mix  
139462                        False           purebred  
       user_id  is_breeder  health_state_old  health_state_new  \
6648      9160        True                 6                 2  

,user_id,email,is_breeder,has_on_target_breed_ancestry,breed_mix_scenario,health_state_old,health_state_new,n_dogs
0,3548.0,cthiem@sbcglobal.net,0.0,False,mix,3.0,2.0,2.0
1,4712.0,desireeryanwood@yahoo.com,1.0,False,mix,3.0,2.0,1.0
2,4712.0,desireeryanwood@yahoo.com,1.0,False,purebred,3.0,2.0,2.0
3,9160.0,jacobhoward537@gmail.com,1.0,False,mix,3.0,2.0,1.0
4,9160.0,jacobhoward537@gmail.com,1.0,False,mix,6.0,2.0,1.0
5,9740.0,jim@bearcreekbluff.com,0.0,False,purebred,3.0,2.0,2.0
6,13268.0,scottswolves@yahoo.com,1.0,False,purebred,3.0,2.0,2.0
7,20453.0,jenpace_anderson@yahoo.com,1.0,False,mix,3.0,2.0,2.0
8,24329.0,bioelf@mindspring.com,0.0,False,purebred,3.0,2.0,4.0
9,40292.0,goodwilliec@ecu.edu,0.0,False,mix,3.0,2.0,2.0


('False', 'mix', 3.0, 2.0) 23
('False', 'mix', 6.0, 2.0) 1
('False', 'purebred', 3.0, 2.0) 13


In [25]:
# calculate genotype freqs in purebred italian greyhounds
all_merged_data_df[
    (all_merged_data_df["has_on_target_breed_ancestry"]=='True')
    &(all_merged_data_df["breed_mix_scenario"]=='purebred')
][["genotype_id", "genotype_id"]].to_csv("purebred_italian_greyhounds_keepfile.txt", sep=' ', index=False, header=False)

chip_probes_list = new_health_id_probe_df_dict[5]

plink.run_plink_with_options(
    [
        "--bfile", "/Users/andreaslavney/Desktop/allGenotypesAllMarkers/allGenotypesAllMarkers",
        "--snps", f"{','.join(chip_probes_list)}",
        "--keep", "purebred_italian_greyhounds_keepfile.txt",
        "--freqx",
        "--out", f"./purebred_italian_greyhounds_new_probes_for_070000",
    ]
)
